# Open Street Map Data Wrangling Project
## Anthony "Ant" Cassetta


## Intro:
The following represents the programmatic cleaning of open street map data from raw xml format into suitable csv files to be used in a sqlite3 schema. 
Please not this is an illustrative example and thus is not exhaustive in all aspects of cleaning the raw data.

### Import statments

In [1]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus
import schema
from collections import defaultdict

#subset sample of data used to quickly validate cleaning the data 
OSMSample = "Cam_sample.osm"

#full data set used for final cleaning execution.
OSMFILE = "Cambridge_osm.osm"

#file connection
osm_file = open("Cambridge_osm.osm", "r", encoding = 'UTF-8')

### Count of tags

In [2]:
'''Counts the tags of the XML file'''
def count_tags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename):
            
        if elem.tag in tags:
            #print('incrementing {}'.format(elem.tag))
            tags[elem.tag] = tags[elem.tag] + 1
        else:
            #print('new tag found {}'.format(elem.tag))
            tags[elem.tag] = 1
        
    return tags

'''Pretty print out of XML tag counts'''
def auditTagCnt(givenFile):

    tags = count_tags(givenFile)
    pprint.pprint(tags)

if __name__ == '__main__':
    auditTagCnt(OSMFILE)

{'member': 27004,
 'nd': 429201,
 'node': 348111,
 'osm': 1,
 'relation': 404,
 'tag': 164293,
 'way': 56539}


### Audit Street Names and State codes to find bad values

In [3]:
street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

state_type_re = re.compile(r'MA',)
state_types = defaultdict(int)

'''Audits the given street name for a fully qualified type. Ex: bad "st" good "Street"'''
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1 

        
'''Audits the given state code Ex: bad "Mass" good "MA"'''
def audit_state_type(street_types, state_name):
    #m = state_type_re.search(state_name)
    #if not m:
        #state_type = m.group()

        state_types[state_name] +=1

        
'''Pretty print out of a given dictonary'''
def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print( "%s: %d" % (k, v)) 


'''check is the given element holds a state attribute'''
def is_state_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:state")


'''check is the given element holds a street attribute'''
def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")


'''takes a given file and audits each element containing a street or state attribute'''
def audit(file):
    for event, elem in ET.iterparse(file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])
        
        elif is_state_name(elem):
            audit_state_type(state_types, elem.attrib['v'])
        
    print('------------Street------------')        
    print_sorted_dict(street_types)
    print('\n---------------State------------')
    print_sorted_dict(state_types)


if __name__ == '__main__':
    audit(OSMFILE)

------------Street------------
#501: 1
1702: 1
6: 1
Ave: 73
Ave.: 14
Avenue: 294
Boulevard: 6
Boylston: 2
Broadway: 42
Cambrdige: 1
Center: 8
Circle: 1
Court: 5
Ct: 2
Dr: 1
Drive: 31
Ext: 1
Fenway: 2
floor: 1
Greenway: 2
Hall: 1
Hampshire: 1
Highway: 2
Holland: 1
Hwy: 1
Lane: 5
Mall: 3
Newbury: 1
Park: 10
Parkway: 8
Pkwy: 5
Place: 30
place: 1
Plaza: 1
Rd: 5
rd.: 1
Road: 39
Row: 3
South: 2
Sq.: 1
Square: 30
St: 113
ST: 1
St,: 1
St.: 20
Street: 985
Terrace: 4
Way: 11
Wharf: 4
Windsor: 1

---------------State------------
MA: 882
ma: 1
Ma: 2
MA- MASSACHUSETTS: 32
Massachusetts: 6
MASSACHUSETTS: 1


### Declare map for cleaning street values

In [4]:
# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street",
            "St,": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Rd.": "Road",
            "Rd": "Road",
            "rd.": "Road",
            "Ct": "Court",
            "Dr": "Drive",
            "Ext": "Extension",
            "Hwy": "Highway",
            "Pkwy": "Parkway",
            "sq": "Square"
            }

### Update methods for dirty attributes

In [5]:
'''Takes a given street name and compares its street type against the dictoary keys. 
If an update is needed an update is made, else none is made.'''
def update_name(name, mapping):
    new_name = ""
    m = street_type_re.search(name)
    if m:
        street_type = m.group()
        #print(street_type)
        if street_type in mapping.keys():
            new_name = re.sub(street_type_re, mapping[street_type], name, count=0, flags=0)
            print('updated ' + name, 'to ' + new_name)
            name = new_name
        #print(new_name)
    return name


'''Ups the state code values.'''
def update_state(state):
    state = 'MA'
    return state


In [6]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

OSM_PATH = OSMFILE

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

'''loads a new XML tag and parses for bad values'''
def load_new_tag(element, secondary, default_tag_type):
    """
    Load a new tag dict to go into the list of dicts for way_tags, node_tags
    """
    new = {}
    #Take the Node element id.
    new['id'] = element.attrib['id']
    
    #check for colon ':' if none, add value. else, parse before and after  
    if ":" not in secondary.attrib['k']:
        new['key'] = secondary.attrib['k']
        new['type'] = default_tag_type
    else:
        post_colon = secondary.attrib['k'].index(":") + 1
        new['key'] = secondary.attrib['k'][post_colon:]
        new['type'] = secondary.attrib['k'][:post_colon - 1]
    
    new['value'] = secondary.attrib['v']
    
    #print secondary.attrib['v']
    return new


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # YOUR CODE HERE
    if element.tag == 'node':
        
        #For each key, value pair of the element attrib dict
        for attrib, value in element.attrib.items():
            if attrib in node_attr_fields:
                node_attribs[attrib] = value
        
        # for elements within the top element
        for secondary in element.iter():
            if secondary.tag == 'tag':
                
                # clean target dirty data
                if is_street_name(secondary):
                    secondary.attrib['v'] = update_name(secondary.attrib['v'], mapping)
        
                if is_state_name(secondary):
                    secondary.attrib['v'] = update_state(secondary.attrib['v'])
                
                #If there is no problemChar parse the value
                if problem_chars.match(secondary.attrib['k']) is None:
                    new = load_new_tag(element, secondary, default_tag_type)
                    tags.append(new)
                    
        
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        
        #For each key, value pair of the element attrib dict
        for attrib, value in element.attrib.items():
            if attrib in way_attr_fields:
                way_attribs[attrib] = value
                
        counter = 0
        for secondary in element.iter():
            
            if secondary.tag == 'tag':
                
                # clean target dirty data
                if is_street_name(secondary):
                    secondary.attrib['v'] = update_name(secondary.attrib['v'], mapping)
                
                if is_state_name(secondary):
                    secondary.attrib['v'] = update_state(secondary.attrib['v'])
                    
                #If there is no problemChar parse the value
                if problem_chars.match(secondary.attrib['k']) is None:
                    new = load_new_tag(element, secondary, default_tag_type)
                    tags.append(new)
            
            if secondary.tag == 'nd':
                newnd = {}
                newnd['id'] = element.attrib['id']
                newnd['node_id'] = secondary.attrib['ref']
                newnd['position'] = counter
                counter += 1
                way_nodes.append(newnd)
        
        
        # print {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w', encoding = 'utf-8') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w', encoding = 'utf-8') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w', encoding = 'utf-8') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w', encoding = 'utf-8') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w', encoding = 'utf-8') as way_tags_file:

        nodes_writer = csv.DictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = csv.DictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = csv.DictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = csv.DictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = csv.DictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    process_map(OSM_PATH, validate=True)


updated Somerville Ave. to Somerville Avenue
updated Somerville Ave to Somerville Avenue
updated Somerville Ave to Somerville Avenue
updated Somerville Ave to Somerville Avenue
updated Cambridge St to Cambridge Street
updated Somerville Ave. to Somerville Avenue
updated Somerville Ave. to Somerville Avenue
updated Walnut St, to Walnut Street
updated Somerville Ave. to Somerville Avenue
updated Somerville Ave. to Somerville Avenue
updated Somerville Ave. to Somerville Avenue
updated Somerville Ave. to Somerville Avenue
updated Somerville Ave. to Somerville Avenue
updated Somerville Ave. to Somerville Avenue
updated Somerville Ave to Somerville Avenue
updated Somerville Ave to Somerville Avenue
updated Main St. to Main Street
updated Holton St to Holton Street
updated Western Ave to Western Avenue
updated Holton St to Holton Street
updated Western Ave to Western Avenue
updated Holton St to Holton Street
updated Holton St to Holton Street
updated Holton St to Holton Street
updated Western

updated Western Ave to Western Avenue
updated Richardson St to Richardson Street
updated Arsenal St to Arsenal Street
updated Holton St to Holton Street
updated Waverly St to Waverly Street
updated Portsmouth St to Portsmouth Street
updated Duval St to Duval Street
updated Duval St to Duval Street
updated Waverly St to Waverly Street
updated Everett St to Everett Street
updated Waverly St to Waverly Street
updated Waverly St to Waverly Street
updated Portsmouth St to Portsmouth Street
updated Waverly St to Waverly Street
updated Waverly St to Waverly Street
updated Athol St to Athol Street
updated Commonwealth Ave to Commonwealth Avenue
updated Birmingham Pkwy to Birmingham Parkway
updated Duval St to Duval Street
updated Birmingham Pkwy to Birmingham Parkway
updated Marshall St. to Marshall Street
updated Boylston St. to Boylston Street
updated Waverly St to Waverly Street
updated Birmingham Pkwy to Birmingham Parkway
updated Litchfield St to Litchfield Street
updated Washington Ave t